# Vaccine Temp pipeline objectives
Notes:

Report ID = [38,39] [2137, 2111]

A:
1) export report: "Export facility info for temperature logs upload", #38 (2137)
2) add field: "redcap_event_name" = "facility_arm_1", for all records
3) add field: "facility_info_complete" = 2, for all records
4) set field "redcap_event_name" as second index
5) import into PID278

B:
1) export report: "Export storage info for temperature logs upload", #39 (2111)
2) add field: "redcap_event_name" = "month_year_arm_1", month spelled out, all lower case, 4 digit integer for year
3) add field: "storage_info_complete" = 2, for all records
4) rename columns according to:

{
 "waiis_fac_name":"waiis_fac_name_s",
 "loc_vfc_pin":"loc_vfc_pin_s",
 "org_name":"org_name_s"
 }

5) set field "redcap_event_name" as second index
6) import into PID278


In [1]:
import redcap

In [2]:
%run -i -n "credentials.py"

In [3]:
print(dev_url)
print(prod_url)

print(["dev_pid100", "dev_pid278", "prod_pid100", "prod_pid278"])

https://qa-redcap.doh.wa.gov/api/
https://redcap.doh.wa.gov/api/
['dev_pid100', 'dev_pid278', 'prod_pid100', 'prod_pid278']


In [4]:
prod100 = redcap.Project(prod_url, prod_pid100)
dev100 = redcap.Project(dev_url, dev_pid100)

In [5]:
dev278 = redcap.Project(dev_url, dev_pid278)

# A) Report: Export Facility info for temperature logs upload, #38 (2137)

## Export Reports and Transform

In [ ]:
# 1)Export Report #38 prod
df_facility_info = prod100.export_report(report_id=2137 ,format_type = "df")
# 2)Create new event field, set all values to "facility_arm_1"
df_facility_info["redcap_event_name"] = "facility_arm_1"
# 3) Create new field for complete/incomplete tag, and set all values to 2
df_facility_info["facility_info_complete"] = 2


In [ ]:
#4) move "redcap_event_name" to first column in prep for import
col = df_facility_info.pop("redcap_event_name")
df_facility_info.insert(0, col.name, col)

In [ ]:
df_facility_info = df_facility_info.reset_index()

In [ ]:
#5) import into dev environment
dev278.import_records(df_facility_info, import_format = "df")

In [ ]:
df_facility_info

# B) Report: Export storage info for temperature logs upload: #39 (2111)

## Export Reports and Transform

In [6]:
# examples: "march_2022_arm_1", "april_2022_arm_1", "may_2022_arm_1", "june_2022_arm_1"
unique_event = "may_2022_arm_1"

In [7]:
# 1)Export Report #38 prod
df_storage_info = prod100.export_report(report_id=2111 ,format_type = "df")

In [8]:
# 2)Create new event field, set all values to "facility_arm_1"
df_storage_info["redcap_event_name"] = unique_event

In [9]:
# 3) Create new field for complete/incomplete tag, and set all values to 2
df_storage_info["storage_info_complete"] = 2

In [10]:
# 4) rename columns 
mapper = {"waiis_fac_name":"waiis_fac_name_s", 
             "loc_vfc_pin":"loc_vfc_pin_s",
             "org_name":"org_name_s" }

df_storage_info = df_storage_info.rename(columns = mapper)

In [11]:
# 5) move "redcap_event_name" to first column in prep for import
col = df_storage_info.pop("redcap_event_name")
df_storage_info.insert(0, col.name, col)

In [12]:
# 6) conver float fields to int
float_fields = df_storage_info.dtypes[df_storage_info.dtypes == float].index
df_storage_info.loc[:, float_fields] = df_storage_info.loc[:, float_fields].astype("Int64")

In [13]:
df_storage_info = df_storage_info.reset_index()

In [15]:
#5) import into dev environment
dev278.import_records(df_storage_info, import_format = "df")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)